In [12]:
from vpython import *
import numpy

scene = canvas(width=800, height=800)

#Classes
# A class is a template for an object
    #Ray class is a template for a ray
    #init: Set all of the variables for the beam
    #define methods

class ray():
    '''A light ray class'''
    def __init__(self,startPoint,directionVector,color):
        self.startPoint = startPoint
        self.oldPoint = startPoint   ### Position of light ray prior to current time step ###
        self.currentPoint = startPoint
        self.directionVector = directionVector.norm()
        self.color = color
        self.length = 10
        self.world_n=1   ### Value of n for medium ray is currently in ###
        self.ray = curve(pos=self.currentPoint, color=self.color)
        
        #self.draw_ray()  ### Commented out to let world() draw ray
        
        #Draw all rays in Source
    def draw_ray(self):
        '''Draw a single light ray'''
        self.oldPoint=self.currentPoint ### Set old point to the current position BEFORE the update ###
        newPoint = self.currentPoint + self.length*self.directionVector
        self.currentPoint = newPoint
        self.ray.append(newPoint)
        
            
        #Loop to check the position of newPoint.  If position is greater than 5, call Snell's Law
        #                                         and assign a new direction vector
    
        #Define normal as normal vector to object
        #Using Snell's Law (numpy.arcsin((1/(1.5))*sin(vector.diff_angle(ray1,normal))))
    def newDirection(self,newDirection):
        '''Set a new direction for the ray'''
        self.directionVector = newDirection                   
        
        #Stop when encounters an object and then reset length to 10
    def newLength(self,newLength):
        '''Set a new length for the vector'''
        self.length=newLength
        
class beam():
    '''A Beam Class'''
    def __init__(self, centerRay, width):
        self.centerRay = centerRay
        self.currentPoint = centerRay.currentPoint
        self.beamDirection = centerRay.directionVector
        self.width = width
        self.color = centerRay.color
        self.beam = []
        ### Begin changes to fix perpendicular rays ###
        self.find_perp()   #Find two rays perpendicular to centerRay
        self.draw_beam()
          
    def find_perp(self):
        '''Find two vectors perpendicular to the center ray'''
        if centerRay.directionVector.y==0 and centerRay.directionVector.x==0:
            self.perp1=vec(1,0,0)
        else:
            self.perp1 = norm(vec(centerRay.directionVector.y,-centerRay.directionVector.x,0))
        self.perp2 = norm(vec(centerRay.directionVector.cross(self.perp1)))
        ### End changes to fix perpendicular rays ###

    def draw_beam(self):
        '''Draw a beam; lots of parallel rays'''
        for i in numpy.arange(0,self.width,.1): 
            for k in numpy.arange(0,2*pi+0.1,.1):
                self.newStart = centerRay.startPoint + i*(self.perp1*cos(k)+self.perp2*sin(k))
                beamRay = ray(self.newStart, self.beamDirection, self.color)                  
                self.beam.append(beamRay)
        
class pointSource():
    def __init__(self, position):
        self.position = position
        self.color = centerRay.color
        self.pointSource = []
        self.source = sphere(pos=vec(centerRay.startPoint), radius=0.01, color=self.color)
        
        self.draw_pointSource()
    
    def draw_pointSource(self):
        '''Draw a point source'''
        self.source
        
        for i in numpy.arange(0,2*pi+0.1,(pi/36)):
            for k in numpy.arange(0,2*pi+0.1,(pi/36)):
                
                x = centerRay.length*(cos(k)*cos(i))
                y = centerRay.length*(cos(k)*sin(i))
                z = centerRay.length*(sin(k))
                
                pointSourceRay = ray(self.position, vec(x,y,z), self.color)
                
                self.pointSource.append(pointSourceRay)
                

      
#####
# Start of World
#####

class world():
    def __init__(self):
        self.rays=[]
        self.objects=[]
        self.dL = 0.1   #Ray step size
        self.n = 1   #Default index of refraction for the world
        self.MAX_LENGTH = 20  ### Set to 10 ###
        self.current_length = 0 ### Length of current ray being drawn ###
        
        
    def add_ray(self,new_ray):
        '''Add a ray to the world'''
        self.rays.append(new_ray)
        new_ray.length=self.dL
        new_ray.world_n = self.n
        
    def add_object(self,new_object):
        '''Add a new object to the world'''
        self.objects.append(new_object)
        
    def draw_rays(self):
        #Loop for ceching Boundaries
        for i in self.rays:
            self.current_length=0
            while self.current_length < self.MAX_LENGTH:  ### Runs until we reach a max length
                for j in self.objects:  ### Loop through all objects ###
                    j.check_boundaries(i)   ### Call check_boundaries method for each object ###
                i.draw_ray()
                self.current_length=self.current_length+self.dL

        #Loop
            
    #def draw_objets(self):

#######
#ball class
######

class ball():
    def __init__(self,position,radius):
        self.pos = position
        self.r = radius
        self.color=color.red
        self.n = 1.5  #Let's make it out of glass for now
        self.shape = 'sphere'
        self.sphere = sphere(pos=self.pos, radius=self.r, color=self.color, opacity=0.1)
        
        
    def surface_normal(self,ray):
        '''Calculate the surface normal at the point a ray hits the sphere'''
        return norm(ray.currentPoint - self.pos)
                
            
    def check_boundaries(self,ray):
        surface_norm = self.surface_normal(ray)
        rel_pos = self.pos - ray.currentPoint
        rel_old_pos=self.pos - ray.oldPoint
        ### The following code should be cleaned up and better commented by me ###
        
        if abs(rel_pos.dot(surface_norm))< self.r and abs(rel_old_pos.dot(surface_norm))> self.r:  ### Going into sphere ###
            thetaIncident = pi-diff_angle(ray.directionVector,surface_norm)
            thetaRefracted = asin((ray.world_n/self.n)*sin(thetaIncident))
            a1 = ray.directionVector - surface_norm * (dot(ray.directionVector,surface_norm))
            ray.directionVector = (-1*cos(thetaRefracted)*surface_norm) + (sin(thetaRefracted)*a1.norm())
        
        if abs(rel_pos.dot(surface_norm))> self.r and abs(rel_old_pos.dot(surface_norm))< self.r:  ### Going out of sphere ###
            thetaIncident = pi-diff_angle(ray.directionVector,surface_norm)
            thetaRefracted = asin((self.n/ray.world_n)*sin(thetaIncident))
            a1 = ray.directionVector - surface_norm * (dot(ray.directionVector,surface_norm))
            ray.directionVector = (1*cos(thetaRefracted)*surface_norm) + (sin(thetaRefracted)*a1.norm())

class lens():
    '''The lens class creates a thin lens with two spherical surfaces'''
    def __init__(self,position,radius):
        self.pos = position
        self.r = radius  #Radius of lens itself
        self.thickness = 1  #Thickness of lens
        self.R1 = 100   #Radius of sphere making up first surface
        self.pos1 = vec(-100,0,0) #Center of first surface sphere
        self.R2 = 100  #Radius of sphere making up second surface
        self.pos2 = vec(100,0,0) #Center of second surface sphere
        self.color=color.red
        self.n = 1.5  #Let's make it out of glass for now
        self.shape = 'lens'
        self.sphere = sphere(pos=self.pos, radius=self.r, color=self.color, opacity=0.1)
        
        
    def surface_normal(self,ray):
        '''Calculate the surface normal at the point a ray hits the sphere'''
        return norm(ray.currentPoint - self.pos)
                
            
    def check_boundaries(self,ray):
        surface_norm = self.surface_normal(ray)
        rel_pos = self.pos - ray.currentPoint
        rel_old_pos=self.pos - ray.oldPoint
        ### The following code should be cleaned up and better commented by me ###
        
        if abs(rel_pos.dot(surface_norm))< self.r and abs(rel_old_pos.dot(surface_norm))> self.r:  ### Going into sphere ###
            thetaIncident = pi-diff_angle(ray.directionVector,surface_norm)
            thetaRefracted = asin((ray.world_n/self.n)*sin(thetaIncident))
            a1 = ray.directionVector - surface_norm * (dot(ray.directionVector,surface_norm))
            ray.directionVector = (-1*cos(thetaRefracted)*surface_norm) + (sin(thetaRefracted)*a1.norm())
        
        if abs(rel_pos.dot(surface_norm))> self.r and abs(rel_old_pos.dot(surface_norm))< self.r:  ### Going out of sphere ###
            thetaIncident = pi-diff_angle(ray.directionVector,surface_norm)
            thetaRefracted = asin((self.n/ray.world_n)*sin(thetaIncident))
            a1 = ray.directionVector - surface_norm * (dot(ray.directionVector,surface_norm))
            ray.directionVector = (1*cos(thetaRefracted)*surface_norm) + (sin(thetaRefracted)*a1.norm())
  
#######
# Test Code
#######

round_thing=ball(vec(1,0,0), 3) 
light1=ray(vec(-5,1,0),vec(1,0,0),color.blue)
light2=ray(vec(-5,-1,0),vec(1,0,0), color.blue)
light3=ray(vec(-5,0,0), vec(1,0,0), color=color.green)
light4=ray(vec(-5,0,1), vec(1,0,0), color=color.blue)
light5=ray(vec(-5,0,1.5), vec(1,0,0), color=color.yellow)
light6=ray(vec(-5,0,-1.5), vec(1,0,0), color=color.yellow)
light7=ray(vec(-5,1.5,0), vec(1,0,0), color=color.yellow)

my_world=world()
my_world.add_ray(light1)
my_world.add_ray(light2)
my_world.add_ray(light3)
my_world.add_ray(light4)
my_world.add_ray(light5)
my_world.add_ray(light6)
my_world.add_ray(light7)
my_world.add_object(round_thing)
my_world.draw_rays()


<IPython.core.display.Javascript object>

In [17]:
from vpython import *

scene=canvas()

ellipsoid(pos=vec(0,0,0), length=1, height=10, width=10)

<IPython.core.display.Javascript object>